In [131]:
import pandas as pd
from glob import glob ## to search and list downloaded files

import numpy as np
import subprocess

from pandasql import sqldf ## make possible to run SQL statements over pandas dataframe equally a PostgreSQL or equivalent environment

## Function to scrape data from Internet giving a URL source

In [159]:
def runcmd(cmd, verbose = False, *args, **kwargs):

    process = subprocess.Popen(
        cmd,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        text = True,
        shell = True
    )
    std_out, std_err = process.communicate()
    if verbose:
        print(std_out.strip(), std_err)
    pass

## Function to unzip downloaded data to be processed

In [ ]:
def unzipDownloadData():
    pass

## Functions to map attributes from html files and processed to a pandas dataframe

In [158]:
f= open("log.txt",'w')
f.close()

f= open("error.txt",'w')
f.close()
#add arquivo de log com os erros

def get_book(text):
    try:
        return text.split("            <h1>")[1].split('</h1>')[0]
    except:
        return np.nan

def get_price(text):
    try:
        return float(text.split('£')[1].split('</p>')[0])
    except:
        return np.nan

def get_rating(text):
    try:
        rating = text.split('star-rating ')[1].split('">')[0]
        
        if  (rating == "One"): return 1
        elif(rating == "Two"): return 2
        elif(rating == "Three"): return 3
        elif(rating == "Four"): return 4
        elif(rating == "Five"): return 5
    
    except:
        return np.nan	
    
def get_copies(text):
	try:
		return int(text.split(' In stock (')[1].split(' available)')[0])
	except:
		return np.nan	
		
def get_info_book(bookInput):
    try:
        with open(bookInput, "r", encoding='utf-8') as f:
            text= f.read()

            book = get_book(text)
            price = get_price(text)
            rating = get_rating(text)
            copies = get_copies(text)
        
    except Exception as e:

        f= open("error.txt",'a')
        f.write(str(e))
        f.write("\n")
        f.close()


        f= open("log.txt",'a')
        f.write(bookInput)
        f.write("\n")
        f.close()
        return False

    return [book, price, rating, copies]

## Processing data from scraped files

In [157]:
files = glob("booksDataWeb/books.toscrape.com/catalogue/*/index.*")
books = []
            
for f in files:
    book = get_info_book(f)
    if book:
        books.append(book)

booksDF = pd.DataFrame(data=books, columns=["BookTitle", "Price", "Rating", "Availability"])
display(booksDF)

,BookTitle,Price,Rating,Availability
0,10-Day Green Smoothie Cleanse: Lose Up to 15 P...,49.71,5,10
1,"10% Happier: How I Tamed the Voice in My Head,...",24.57,2,10
2,"1,000 Places to See Before You Die",26.08,5,1
3,11/22/63,48.48,3,11
4,13 Hours: The Inside Account of What Really Ha...,27.06,1,13
...,...,...,...,...
995,You can&#39;t bury them all: Poems,33.63,2,17
996,You (You #1),43.61,5,14
997,Zealot: The Life and Times of Jesus of Nazareth,24.70,3,14
998,Zero History (Blue Ant #3),34.77,1,3


## What is the average price of books by rating (in stars)?

In [154]:
def avgPrinceByRating(booksTable):
    
    result = sqldf("""SELECT Rating, ROUND(AVG(price),2) as "AVG Price (£)" 
                          FROM booksTable 
                          GROUP BY Rating;""")
    
    return result

## Call function given the dataframe 
avgPrinceByRating(booksDF)

,Rating,AVG Price (£)
0,1,34.56
1,2,34.81
2,3,34.69
3,4,36.09
4,5,35.37


## How many books have 2 or less copies on a specific day?

In [156]:
def amountCopiesBooks(booksTable):
    result = sqldf("""SELECT COUNT(*) as "Books have 2 or less copies" 
                        FROM booksTable 
                        WHERE Availability <=2 
                        """)
    
    return result

## Call function given the dataframe 
amountCopiesBooks(booksDF)

,Books have 2 or less copies
0,112
